In [2]:

'''
File built on top of BigramModel file which contains 
transforer code and explaination.

Above was Transformer but we will look into GPT (Generative Pre-trained Trasformer). where only Decoder part stay.. and not the Encoder nor the 2nd unmasked Multihead attention (refer research paper image)
Before this, lets look into Self Attention

Decoder only structure (GPT Structure)

Traditional Inputs ---> Embeddings+positional encoding ---> decoders X N one after another (sequential)--->nnLinear --->softmax --->samplea dn generating output and compare to imput to optimize
Within each decoder: We have :
Multihead attention ---> residual connection+norm --->feed forward ---> residual connect+norm

Multihead Attention part in Decoder:
-Improtant: imagine different people(heads) learning same thing from different perspective.
-all heads have different learnable parameters.
-Keys (K), Query(Q) and Value(V) in each head, togethere go into Scaled Dot product attention ---> concat results ---> Linear Layer
1. K, Q and V are essense of self attention models. (explained later)
2. Scaled dot Prod. Attention: there are N such heads with respective K, Q and Vs which work and give N outputs
3. Hence we need concat results section which then is passed to nn.linear 

Scaled Dot Prod Attention Part in Multihead Attention:
-Key(K) emits different tensor for every token in sentence
-Query(Q) says, what am i looking for ? for some important words, dot product of K and Q will be very high, unlike some other words like "A, an, The etc."
1. we now scale this dot prod by 1/ sqrt (len of row in keys) helps prevent dot products explode.
2. This scaled value is now fed to torch.tril() which helps learn from the tokens we have already seen while keeping ones ahead = 0 (look at Tril matrix, ull understand..). This process is called Attention Masking.
3. we then apply softmax to above output. why? because it will then give important to the few attentions that matter more as comapred to other. This helps increase confidence of the model.
4. matrix multipy it with original Value(V) to retain info if lost in computation.


Important:
Key points to note on how transformer is different from GPT below.
1. Transformers have embeddings as we have seen (nn.embedding) which has its dimentions for every token in consideration. there is although no way to transmit "positional importance" wrt previous and next tokens.
2. Positional encodding is a method to push some positional patterns so that model can consider them in its calculations.
   - This is done by creating a positional encoding (PE) vector for every dimension a token has. in our case, dim. = vocab_size.
   
   eg. for 5th character, having 87 dimensional embedding vector now have PE:
   PE(5, 0=dim) = sin(pos/(10000^(0/87)))
   PE(5, 1=dim) = cos(pos/(10000^(1/87)))
   PE(5, 2=dim) = sin(pos/(10000^(2/87)))
   PE(5, 3=dim) = cos(pos/(10000^(3/87)))   .. and so on till  87.
   Note the alternating Sine and cosine.
   Even Dimensions (0, 2, 4, ...): These dimensions undergo a sine transformation. For a given token, every even dimension will have its value computed using the sine function.
   Odd Dimensions (1, 3, 5, ...): These dimensions undergo a cosine transformation. For a given token, every odd dimension will have its value computed using the cosine function.

3. This way, eg. PE(5, 0=dim) has not built a mathematical relation with PE(4, 0=dim) and PE(6, 0=dim) so on for each dimension.
4. PE is restricted to number of tokens considered in a block. For us, we have considered 8, so math tries to find relations only within these 8 tokens in one block.


'''

'\nFile built on top of BigramModel file which contains \ntransforer code and explaination.\n\nAbove was Transformer but we will look into GPT (Generative Pre-trained Trasformer). where only Decoder part stay.. and not the Encoder nor the 2nd unmasked Multihead attention (refer research paper image)\nBefore this, lets look into Self Attention\n\nDecoder only structure (GPT Structure)\n\nTraditional Inputs ---> Embeddings+positional encoding ---> decoders X N one after another (sequential)--->nnLinear --->softmax --->samplea dn generating output and compare to imput to optimize\nWithin each decoder: We have :\nMultihead attention ---> residual connection+norm --->feed forward ---> residual connect+norm\n\nMultihead Attention part in Decoder:\n-Improtant: imagine different people(heads) learning same thing from different perspective.\n-all heads have different learnable parameters.\n-Keys (K), Query(Q) and Value(V) in each head, togethere go into Scaled Dot product attention ---> concat 

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import mmap
import random
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 32
batch_size = 128       #Each batch has 4 blocks under them
max_iters = 200
eval_iters = 100       #To print loss updates
lr = 3e-4              #very common ones to try are:3e-3, 1e-3, 3e-4, 1e-3
n_head = 1
n_embd = 384         #similar to waht we used in BigramModel(vocab_size) ie. Vector size. 
X_layers = 1         #No of Sequential decoder Blocks

cpu


In [6]:
with open('openwebtext/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# creating vocabulary
chars = sorted(set(text))  # len = 81
vocab_size = len(chars)

# now we tokenize them - basically giving them an incremental integer numbers to address them
str_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_str = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [str_to_int[c] for c in s]          #encode("Hello")
decode = lambda I: ''.join([int_to_str[i] for i in I]) #decode([12, 45, 2, 78])

In [8]:
data = torch.tensor(encode(text), dtype = torch.long)

n = int(0.8*len(data))
train_data = data[:n]
valid_data = data[n:]

def get_random_chunk(split):
    filename = "Output/output_train.txt" if split == "train" else "Output/output_val.txt"
    with open(filename, 'rb') as f:      #as the file is 37.42GB, it is too long for a simple open() func to carry. we need to use memory Map reading
        with mmap.mmap(f.fileno(), 0, access = mmap.ACCESS_READ) as mappedFile:  #mmap helps bring file straight to RAM sacrificing sapce for time to help you read chunk without opening whole file. f.fileno(): RAM numbers eevry open file. this command helps link mmap func to that file. 0: indicating that the system should create a memory map large enough to hold the entire file. access for mmap should match access fiven to open()
            size = len(mappedFile)
            start_pos = random.randint(0, size-block_size*batch_size)
            #go to start postion now
            mappedFile.seek(start_pos)
            chunk = mappedFile.read(block_size*batch_size-1)  # -1 is precaution to avoid overflow
            
            #block we got is encoded. we need to decode it
            word_block = chunk.decode('utf-8', errors='ignore').replace('\r', '')    #errors=ignored doesnt stop execution, just ignored the error pathc and considers next one to decode.
            int_data = torch.tensor(encode(word_block), dtype=torch.long)            #we are interested only in intergers associated with each token word/character
    
    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:t+block_size] for i in ix])
    y = torch.stack([data[i+1:t+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch("train")

FileNotFoundError: [Errno 2] No such file or directory: 'Output/output_train.txt'

In [70]:
@torch.no_grad()         #temporarily disable gradient computation when the below function is called.
def estimate_loss():
    out = {}
    model.eval()        # helps diable operations specific to training phase 
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)   # model calculating loss N (we have arbitrarily kept N=eval_iters but it can be any large number to make sure loss is avg over several samples) times and then avg. it out 
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()       # model is set back to training mode
    return out

In [71]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)      
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))      #registers this state of no-look-ahead in a form of a model. helps save computation as we dont need to redo this step later.
        
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # input of size (B, T, C)
        # Output of  (B, T, Head_size)
        
        B, T, C = x.shape
        k = self.key(x)   #(B, T, Head_size)
        q = self.query(x) #(B, T, Head_size)
        
        attn_scores = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5   # (B, T, Head_size) @ (B, Head_size, T) --> (B, T, T)   #transpose(-2,-1) is swapping 2ndlast dim to last dim. the whole line helps moderate values coming out of all heads and no single head is dominant. hence all the voices from all heads is scalled down by sqrt them.
        attn_scores = attn_scores.masked_fill(self.tril[:T, :T] ==0, float('-inf'))    #(B, T, T)   #upper side of triangle in matrix is -inf. Softmax next turns it to 0
        attn_scores = F.softmax(attn_scores, dim=-1)
        attn_scores = self.dropout(attn_scores)
        
        v = self.value(x)
        out = attn_scores @ v
        return out

In [72]:
class MultiHeadAttention(nn.Module):   # multiple heads in parallel
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])     # container class provided by PyTorch to hold a list of PyTorch modules. helps run things in parallel unlike nn.sequential
        self.proj = nn.Linear(head_size * n_head, n_embd)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        out = torch.concat([h(x) for h in self.heads], dim=-1)   # (B, T, F) becomes (B, T, [h1 h1 h1, h2, h2, h2, h3, h3, h3, h3]) where H are features concat. accross last dim.
        out = self.dropout(self.proj(out))
        return out

In [73]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(0.2)
            )
    def forward(self, x):
        return self.net(x)

In [74]:
class DecoderBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head   #num. of features each head is capturing
        self.selfAtten = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)  # for 2 postNorm architectures.
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.selfAtten(x)
        x = self.ln1(x+y)    #we are first adding x and y then applying a Norm on it
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x
        

In [75]:
class GPTLangModel(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super().__init__()                   #The super().__init__() call ensures that the BigramLangModel class correctly inherits the behavior and properties of the nn.Module class. This practice allows you to build complex neural network models while benefiting from the foundation provided by PyTorch's nn.Module.
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)     #here, first dim is vocab. of unique tokens (vocab_length) and 2nd dimention is the length of the dense vector for that index token (embedding dimension)
        #The embedding dimension is a crucial parameter because it determines the capacity of the embedding space to capture and represent information about the tokens. Larger embedding dimensions can potentially capture more complex patterns and relationships in the data but may also require more computational resources.
        self.positional_embedding_table = nn.Embedding(block_size, n_embd)
        #Creating X_layers of sequential (Decoder layer blocks)
        self.blocks = nn.Sequential(*[DecoderBlock(n_embd, n_head=n_head) for _ in range(X_layers)])   # * helps create X No of detatched nn.sequent. blocks 
        self.ln_f = nn.LayerNorm(n_embd)  #creating a checkpoint for the final layer after laster decoder, ie. a Linear Layer. It is not a layer but a technique to normalize values coming from decoder before feeding it into Linear Layer. It applies normalization to the veatures of teach token in the batch and has its own learnable parameters. this step will be removed later (maybe)
        self.ln_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)  #method by nn.Module class to apply weights to all sub-classes and self. apply takes care of which module to pass as parameter hence we dont.
    
    def _init_weights(self, module):    #Private method to init.weights. module is any sub-module(layer) of the nn
        if isinstance(module, nn.Linear):  #checks if module is an instance of Linear or Embedding
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)  #init. weights using normal distr.
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)    #Initializing biases to zeros is a common practice, especially when weights are initialized with small random values.
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):
        B, T = index.shape
        
        tok_embed = self.token_embedding_table(index)                                  # (B, T=block_size, C) similar to logits previously
        pos_embed = self.positional_embedding_table(torch.arange(T, device=device))   # (T, C). performs an embedding lookup. It takes the indices generated by torch.arange and retrieves the corresponding rows from the embedding table.
        # Good practice to add both above embeddings
        X = tok_embed + pos_embed   # becomes (B, T, C) refer "Broadcasting Sementics pytorch" to know how
        X = self.blocks(X)          # (B, T, C)
        X = self.ln_f(X)            # (B, T, C)
        logits = self.ln_head(X)    # (B, T, vocab_size)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape         #B = Batch, T = Time(unknown/next seq. of integers), C = channels/vocab_size
            logits = logits.view(B*T, C)   # .view helps repack into a new shape as per need. we need 2nd parameter to be C, doesnt matter what first is.
            targets = targets.view(B*T)     #helped reduce dimension.
            loss = F.cross_entropy(logits, targets)    # above operation on logits was done as cross_entropy accepts 1st input with shape (X, Channels) and not (X, Y, channels). Targets simmilary should be of shape (X*Y),  i.e only 1 dim
            # Cross-entropy is a measure of the difference between two probability distributions for a given random variable. In the context of machine learning, and particularly in classification tasks, cross-entropy is commonly used as a loss function to measure the difference between the predicted probability distribution and the true probability distribution of the target
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            #predictions
            logits, loss = self.forward(index) # no target given hence "else loop" in forward doesnt run. Loss = none and logit's shape is (dim of index(1,1), vocab_size(ie. 87))
            
            #Focussing only on last Time step and not all of T
            logits = logits[:, -1, :] # to get (B, C). why only last T?
            #softmax
            probs = F.softmax(logits, dim=-1) #focussing only on last dimension of logits here. Output of model is passed to softmax to create probablities
            
            #sample from distribution
            #----Consider the scenario where you are generating text, and at a certain point, the model has multiple plausible options for the next character. If you always choose the most probable character, the generated sequences might become deterministic and lack diversity. By introducing randomness through sampling, you allow the model to explore different paths and create more diverse outputs.
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1) eg. [54]
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        return index
    
model = GPTLangModel(vocab_size, n_embd)
print('loading model...')
with open('model_001.pkl', 'rb') as f:    #helps retrain model automatically
    model = pickle.load(f)

print('success...')
m = model.to(device)

# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars) 

In [76]:
#Dummy code to understand embedding
# more info in: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
word_to_ind = {"hello":0, "world": 1}
embeds = nn.Embedding(2, 10)


lookup_tensor1 = torch.tensor([0], dtype=torch.long)
# print(lookup_tensor.shape)
lookup_tensor2 = torch.zeros((1, 1), dtype=torch.long)
xyz = embeds(lookup_tensor1)
zxy = embeds(lookup_tensor2)
print(xyz)
print(xyz.shape)
print(zxy)
print(zxy.shape)
# Note: we passed a tensor of dim 3. hence output was (3, 10). if we had vocab size of 10,000 and had passed tensor of dim 65, we would have gottent tensor of dim (65, 10))
#these outputs are called logits (not correct values though as they are not trained with training data)

tensor([[-0.4016,  0.0422,  1.0896,  0.2567,  0.2861, -0.5000,  0.2231, -0.4832,
         -0.3609, -0.1560]], grad_fn=<EmbeddingBackward0>)
torch.Size([1, 10])
tensor([[[-0.4016,  0.0422,  1.0896,  0.2567,  0.2861, -0.5000,  0.2231,
          -0.4832, -0.3609, -0.1560]]], grad_fn=<EmbeddingBackward0>)
torch.Size([1, 1, 10])


In [77]:
#Optimizer code
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)   #AdamW applies Decay (penalty) to large weights. Adam Doesn't.

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'iteration: {iter}, Loss: {losses}')
        
    xb, yb = get_batch("train")
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)    #previous grad. calculations are changed to 0. They are usually usefull in RNN model where reference to previous grad calculations is an important idea.
    loss.backward()
    optimizer.step()     #to take step in the right direction
print(loss.item())

with open('model_001.pkl', 'wb') as f:
    pickle.dump(model, f)           #we could have used torch.load and torch.save as always but this is more straight forward and easy

KeyboardInterrupt: 

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)